### 下載模組

在終端機輸入以下指令：

```
pip install selenium
```

### 匯入模組

In [1]:
import os
import base64
import random

# 函式註解
from typing import *

# 資料庫
from firebase import firebase

# 網路爬蟲
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

## 下載資料庫內容

In [2]:
DATABASE_URL = "https://fir-test-9907d-default-rtdb.firebaseio.com/"

fdb = firebase.FirebaseApplication(DATABASE_URL, None)
response = fdb.get("/meme_template", None)

### 定義 Class MemeGenerator 

In [3]:
class MemeGenerator:
    def __init__(self, texts: List[str], url: str) -> None:
        self.url = url
        self.texts = texts

        self.chrome_options = Options()
        self.chrome_options.add_argument("--disable-gpu")

        # 無頭模式
        # self.chrome_options.add_argument("--headless")

    # 字數過長，則加上換行符號
    def texts_preprocessing(self) -> None:
        for index, text in enumerate(self.texts):
            if len(text) > 7:
                # 每 7 個字加上換行符號
                self.texts[index] = '\n'.join([text[i:i+7] for i in range(0, len(text), 7)])
                
    # 開啟瀏覽器
    def open_webdriver(self) -> None:
        # 初始化瀏覽器、設置智能等待
        # 注意!!! implicitly_wait 不要設定得太短
        self.driver = webdriver.Chrome('chromedriver', options = self.chrome_options)
        self.driver.implicitly_wait(20)

        # 開啟瀏覽器，並固定視窗大小
        self.driver.get(self.url)
        self.driver.set_window_size(1200, 800)
        sleep(0.5)

    # 輸入文本
    def typing_texts(self) -> None:
        parent = self.driver.find_element(By.CLASS_NAME, "mm-boxes")

        # 找到所有可填寫的欄位，並逐一填寫文本
        textareas = parent.find_elements(By.CLASS_NAME, "mm-text")
        for index, textarea in enumerate(textareas):
            textarea.send_keys(self.texts[index])

        sleep(0.5)

    # 下載圖片
    def download_img(self, image_path: str) -> None:
        # 點擊下載按鈕
        download_btn = self.driver.find_element(By.XPATH, r"/html/body/div[3]/div[2]/div[2]/div[10]/div[5]/button[1]")
        download_btn.click()

        # 找到圖片，並取得下載連結
        img = self.driver.find_element(By.ID, "done-img")
        img_url = img.get_attribute("src")
        sleep(0.5)

        # 解碼 Base64 圖片資料
        image_data = img_url.split(",")[1]
        image_bytes = base64.b64decode(image_data)

        # 輸出圖片
        with open(image_path, "wb") as file:
            file.write(image_bytes)
        
        sleep(0.5)

    # 關閉瀏覽器
    def close(self) -> None:
        self.driver.quit()

### 使用 Class Meme_Generator 

In [4]:
# 文本，預計用 AI 生成
texts = ["文本 1", "總字數超過7個字就換行", "文本 3", "文本 4", "文本 5", "文本 6", "文本 7", "文本 8"]

# 隨機抽取網址
urls = list(value["url"] for value in response.values())
url = random.choice(urls)

# 輸出梗圖的檔案名稱
result_img_name = "result"

# 進行網路爬蟲，最終結果在 "result_pictures" 資料夾
MemeGen = MemeGenerator(texts, url)
MemeGen.open_webdriver()
MemeGen.texts_preprocessing()
MemeGen.typing_texts()
MemeGen.download_img(f"./result_pictures/{result_img_name}.png")
MemeGen.close()